In [1]:
import pyodbc
import pandas as pd
import requests

# === CONFIG ===
HF_API_TOKEN = "hf_kLrNgZpqdrKpxxxxxxxUOuvtkWxQGwOcexxxxxxxQvmMXAmL" 
MODEL_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"

# === Hugging Face Headers ===
headers = {
    "Authorization": f"Bearer {HF_API_TOKEN}",
    "Content-Type": "application/json"
}

# === Getiing Table Information inside the database ===
def get_table_info(cursor, database):
    """Fetches basic schema info: table names & columns."""
    cursor.execute("""
        SELECT TABLE_NAME, COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_CATALOG = ?
        ORDER BY TABLE_NAME
    """, database)

    rows = cursor.fetchall()
    schema = {}
    for table, column in rows:
        schema.setdefault(table, []).append(column)
    return schema


def build_schema_prompt(schema):
    """Formats schema info into a readable prompt."""
    schema_str = ""
    for table, columns in schema.items():
        schema_str += f"Table: {table}\nColumns: {', '.join(columns)}\n\n"
    return schema_str


def call_huggingface(prompt):
    """Calls HF model with the given prompt."""
    payload = {"inputs": prompt}
    response = requests.post(MODEL_URL, headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()[0]["generated_text"]
    else:
        raise Exception(f"HF Error {response.status_code}: {response.text}")


def run_sql_query(cursor, sql):
    """Runs the generated SQL and returns a DataFrame."""
    try:
        df = pd.read_sql_query(sql, cursor.connection)
        return df
    except Exception as e:
        return f"Error running query: {e}"


def main():
    # Step 1: Connect to SQL Server
    conn_str = (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=NIKHIL\SQLEXPRESS;"
        "DATABASE=AdventureWorksLT;"
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Step 2: Get schema info
    schema = get_table_info(cursor, 'AdventureWorksLT')
    schema_text = build_schema_prompt(schema)

    # Step 3: Ask user for input
    while True:
        user_question = input("Ask a SQL question or 'exit' to end chat:\n> ")
        
        if user_question.lower() in ['exit', 'stop']:
            print("Good Bye...!!!")
            break
            
        # Step 4: Build prompt
        prompt = f"""You are a SQL Server generator AI. Given the following database schema and instruction, generate a valid SQL query.Schema:
        {schema_text}

Instruction: {user_question}
SQL:"""

        # Step 5: Get SQL query from model
        generated = call_huggingface(prompt)
        sql_query = generated.split("SQL:")[-1].strip()

        print("\n🔍 Generated SQL:\n", sql_query)

        # Step 6: Run SQL query
        result = run_sql_query(cursor, sql_query)
        print("\n📊 Query Result:\n", result)

    cursor.close()
    conn.close()


if __name__ == "__main__":
    main()


C:\Users\ADMIN\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Ask a SQL question or 'exit' to end chat:
> Select top 10 records from the SalesLT.Address table

🔍 Generated SQL:
 SELECT TOP 10 *
FROM SalesLT.Address;


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2348\4120746972.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, cursor.connection)



📊 Query Result:
    AddressID         AddressLine1 AddressLine2      City     StateProvince  \
0          9    8713 Yosemite Ct.         None   Bothell        Washington   
1         11  1318 Lasalle Street         None   Bothell        Washington   
2         25     9178 Jumping St.         None    Dallas             Texas   
3         28     9228 Via Del Sol         None   Phoenix           Arizona   
4         32    26910 Indela Road         None  Montreal            Quebec   
5        185      2681 Eagle Peak         None  Bellevue        Washington   
6        297      7943 Walnut Ave         None    Renton        Washington   
7        445   6388 Lake City Way         None   Burnaby  British Columbia   
8        446    52560 Free Street         None   Toronto           Ontario   
9        447    22580 Free Street         None   Toronto           Ontario   

   CountryRegion PostalCode                               rowguid ModifiedDate  
0  United States      98011  268AF621-76D7

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_2348\4120746972.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, cursor.connection)



🔍 Generated SQL:
 SELECT * FROM SalesLT.Address
WHERE AddressID = 9;

📊 Query Result:
    AddressID       AddressLine1 AddressLine2     City StateProvince  \
0          9  8713 Yosemite Ct.         None  Bothell    Washington   

   CountryRegion PostalCode                               rowguid ModifiedDate  
0  United States      98011  268AF621-76D7-4C78-9441-144FD139821A   2006-07-01  
Ask a SQL question or 'exit' to end chat:
> exit
Good Bye...!!!
